In [ ]:
import requests
import os

# 配置文件的URL
config_url = "https://huggingface.co/microsoft/resnet-50/resolve/main/config.json"

# 指定下载路径
save_dir = "/home/wangsheng/Downloads"
os.makedirs(save_dir, exist_ok=True)
config_path = os.path.join(save_dir, "config.json")

# 下载并保存配置文件
response = requests.get(config_url)
response.raise_for_status()  # 确保请求成功

with open(config_path, "wb") as f:
    f.write(response.content)

print(f"Config file has been saved to {config_path}")


In [10]:
import heapq
# 决策单个文件的存储位置
def get_disk(file_size, disk_spaces):
    # 使用贪心算法，获取合适的存储位置代码
    # 将磁盘可用空间和磁盘编号打包成元组，并按可用空间递增排序
    available_spaces = [(space, i) for i, space in enumerate(disk_spaces)]
    heapq.heapify(available_spaces)
    
    # 贪心选择可用空间最大的磁盘
    while available_spaces:
        _, disk_id = heapq.heappop(available_spaces)
        if disk_spaces[disk_id] >= file_size:
            return str(disk_id + 1)  # 磁盘编号从 1 开始

      # 如果没有足够的磁盘空间，则返回 -1 表示无法放置
    return str(-1)

# 测试代码
file_size = 200
disk_spaces = [100, 50, 300, 150]

disk_id = get_disk(file_size, disk_spaces)
print(f"文件应该放置在磁盘 {disk_id} 上")

文件应该放置在磁盘 3 上


In [9]:
import heapq
# 决策文件列表的存储位置
def get_disk(file_sizes, disk_spaces):
# 将磁盘可用空间和磁盘编号打包成元组，并按可用空间递增排序
  available_spaces = [(space, i) for i, space in enumerate(disk_spaces)]
  heapq.heapify(available_spaces)

  # 初始化文件放置情况
  file_placements = {}

  for file_size in file_sizes:
    # 贪心选择可用空间最大的磁盘
    while available_spaces:
      _, disk_id = heapq.heappop(available_spaces)
      if disk_spaces[disk_id] >= file_size:
        file_placements[file_size] = disk_id + 1  # 磁盘编号从 1 开始
        disk_spaces[disk_id] -= file_size
        break

  return file_placements

# 测试代码
file_sizes = [100, 50, 300, 250, 150]
disk_spaces = [200, 150, 300, 100]

file_placements = get_disk(file_sizes, disk_spaces)
print(file_placements)

{100: 4, 50: 2, 300: 3}


In [1]:
def greedy_file_storage(files, disks):
    """
    使用贪心算法实现文件存储位置的决策

    :param files: 文件大小列表
    :param disks: 磁盘剩余空间列表
    :return: 每个文件存储在哪个磁盘上的列表
    """
    # 将文件按照大小从大到小排序，并记录原始索引
    sorted_files = sorted(enumerate(files), key=lambda x: -x[1])
    # 记录每个文件的存储位置，初始化为-1表示未存储
    storage = [-1] * len(files)

    for original_index, file_size in sorted_files:
        # 找到能够容纳该文件且剩余空间最小的磁盘
        best_disk_index = -1
        best_disk_space = float('inf')

        for j, disk_space in enumerate(disks):
            if file_size <= disk_space and disk_space < best_disk_space:
                best_disk_index = j
                best_disk_space = disk_space

        if best_disk_index != -1:
            # 将文件存储到该磁盘上
            disks[best_disk_index] -= file_size
            # 记录存储位置
            storage[original_index] = best_disk_index

    return storage

# 示例输入
files = [4, 8, 1, 4, 2]
disks = [10, 10, 10]

# 调用函数
storage = greedy_file_storage(files, disks)

# 输出结果
print("文件存储位置:", storage)
print("剩余磁盘空间:", disks)


文件存储位置: [1, 0, 1, 1, 0]
剩余磁盘空间: [0, 1, 10]


In [3]:
import paramiko

def get_remote_block_device_space(hostname, port, username, password, device):
    """
    获取远程主机上块设备的存储空间

    :param hostname: 远程主机名或IP地址
    :param port: SSH端口
    :param username: SSH用户名
    :param password: SSH密码
    :param device: 块设备路径
    :return: 块设备的存储空间信息
    """
    # 创建SSH客户端
    client = paramiko.SSHClient()
    # 自动添加SSH密钥
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # 连接远程主机
        client.connect(hostname, port, username, password)
        
        # 执行命令获取块设备信息
        stdin, stdout, stderr = client.exec_command(f'lsblk -b -o NAME,SIZE | grep -w {device}')
        
        # 读取命令输出
        output = stdout.read().decode()
        error = stderr.read().decode()
        
        if error:
            print(f"Error: {error}")
            return None
        
        # 解析输出获取块设备大小
        device_info = output.strip().split()
        if device_info:
            device_name, device_size = device_info
            return {
                "device": device_name,
                "size": int(device_size)
            }
        else:
            print("设备信息未找到")
            return None

    finally:
        # 关闭连接
        client.close()

# 示例使用
hostname = '192.168.0.7'
port = 22
username = 'wangsheng'
password = 'sheng123'
device = 'mmcblk0p1'

device_info = get_remote_block_device_space(hostname, port, username, password, device)
if device_info:
    print(f"设备: {device_info['device']}")
    print(f"大小: {device_info['size']} 字节")


设备: ├─mmcblk0p1
大小: 62090379264 字节


In [5]:
import paramiko

def get_remote_disk_space(hostname, port, username, password, device):
    """
    获取远程主机上块设备的存储空间信息

    :param hostname: 远程主机名或IP地址
    :param port: SSH端口
    :param username: SSH用户名
    :param password: SSH密码
    :param device: 块设备路径
    :return: 块设备的存储空间信息，包括总空间大小、可用空间和已用空间
    """
    # 创建SSH客户端
    client = paramiko.SSHClient()
    # 自动添加SSH密钥
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # 连接远程主机
        client.connect(hostname, port, username, password)
        
        # 执行命令获取块设备信息，使用 df 命令获取指定设备的信息
        stdin, stdout, stderr = client.exec_command(f'df -B1 {device}')
        
        # 读取命令输出
        output = stdout.read().decode()
        error = stderr.read().decode()
        
        if error:
            print(f"Error: {error}")
            return None
        
        # 解析输出获取块设备信息
        lines = output.strip().split("\n")
        if len(lines) > 1:
            headers = lines[0].split()
            values = lines[1].split()
            
            disk_info = {
                "filesystem": values[0],
                "total": int(values[1]),
                "used": int(values[2]),
                "available": int(values[3]),
                "use%": values[4],
                "mounted_on": values[5]
            }
            return disk_info
        else:
            print("设备信息未找到")
            return None

    finally:
        # 关闭连接
        client.close()

# 示例使用
hostname = '192.168.0.7'
port = 22
username = 'wangsheng'
password = 'sheng123'
device = '/dev/mmcblk0p1'

disk_info = get_remote_disk_space(hostname, port, username, password, device)
if disk_info:
    print(f"文件系统: {disk_info['filesystem']}")
    print(f"总空间大小: {disk_info['total']} 字节") 
    print(f"已用空间: {disk_info['used']} 字节")
    print(f"可用空间: {disk_info['available']} 字节")
    print(f"使用率: {disk_info['use%']}")
    print(f"挂载点: {disk_info['mounted_on']}")


文件系统: /dev/mmcblk0p1
总空间大小: 60846690304 字节
已用空间: 42674417664 字节
可用空间: 15070588928 字节
使用率: 74%
挂载点: /


In [2]:
import time

def your_function():
    # 你的函数代码
    time.sleep(2)  # 示例：模拟函数执行时间

start_time = time.time()
your_function()
end_time = time.time()

execution_time = end_time - start_time
print(f"Function execution time: {execution_time} seconds")
print(type(execution_time))

Function execution time: 2.000131130218506 seconds
<class 'float'>


In [1]:
import openai

openai.api_key = 'anything'
openai.base_url = "http://localhost:3040/v1/"

completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "How do I list all files in a directory using Python?"},
    ],
)

print(completion.choices[0].message.content)

TypeError: 'NoneType' object is not subscriptable

In [ ]:

loaded_model_state = torch.load(file_path, map_location='cpu')
            # print(loaded_model_state)
layer_name_list, layer_hash_list = utils._save_state_dict(state_dict=loaded_model_state, path=model_path)

In [2]:
import random

def generate_filepaths_array(num_entries):
    filepaths_array = []
    for i in range(num_entries):
        filepath = f"filepath{i+1}"
        value = random.randint(0, 100)  # 生成随机值
        filepaths_array.append([filepath, value])
    return filepaths_array

filepaths_array = generate_filepaths_array(500)

# 输出前几个数据以确认
print(filepaths_array)

[['filepath1', 29], ['filepath2', 50], ['filepath3', 20], ['filepath4', 96], ['filepath5', 29], ['filepath6', 14], ['filepath7', 47], ['filepath8', 7], ['filepath9', 68], ['filepath10', 48], ['filepath11', 89], ['filepath12', 12], ['filepath13', 7], ['filepath14', 38], ['filepath15', 24], ['filepath16', 21], ['filepath17', 2], ['filepath18', 86], ['filepath19', 57], ['filepath20', 58], ['filepath21', 57], ['filepath22', 2], ['filepath23', 82], ['filepath24', 39], ['filepath25', 1], ['filepath26', 98], ['filepath27', 100], ['filepath28', 97], ['filepath29', 91], ['filepath30', 1], ['filepath31', 78], ['filepath32', 70], ['filepath33', 82], ['filepath34', 70], ['filepath35', 74], ['filepath36', 84], ['filepath37', 25], ['filepath38', 40], ['filepath39', 2], ['filepath40', 40], ['filepath41', 87], ['filepath42', 59], ['filepath43', 47], ['filepath44', 7], ['filepath45', 87], ['filepath46', 81], ['filepath47', 72], ['filepath48', 42], ['filepath49', 41], ['filepath50', 0], ['filepath51', 2

In [15]:
test_array = [['filepath1', 29], ['filepath2', 50], ['filepath3', 20], ['filepath4', 96], ['filepath5', 29], ['filepath6', 14], ['filepath7', 47], ['filepath8', 7], ['filepath9', 68], ['filepath10', 48], ['filepath11', 89], ['filepath12', 12], ['filepath13', 7], ['filepath14', 38], ['filepath15', 24], ['filepath16', 21], ['filepath17', 2], ['filepath18', 86], ['filepath19', 57], ['filepath20', 58], ['filepath21', 57], ['filepath22', 2], ['filepath23', 82], ['filepath24', 39], ['filepath25', 1], ['filepath26', 98], ['filepath27', 100], ['filepath28', 97], ['filepath29', 91], ['filepath30', 1], ['filepath31', 78], ['filepath32', 70], ['filepath33', 82], ['filepath34', 70], ['filepath35', 74], ['filepath36', 84], ['filepath37', 25], ['filepath38', 40], ['filepath39', 2], ['filepath40', 40], ['filepath41', 87], ['filepath42', 59], ['filepath43', 47], ['filepath44', 7], ['filepath45', 87], ['filepath46', 81], ['filepath47', 72], ['filepath48', 42], ['filepath49', 41], ['filepath50', 0], ['filepath51', 26], ['filepath52', 15], ['filepath53', 50], ['filepath54', 50], ['filepath55', 13], ['filepath56', 52], ['filepath57', 70], ['filepath58', 45], ['filepath59', 92], ['filepath60', 52], ['filepath61', 91], ['filepath62', 38], ['filepath63', 78], ['filepath64', 70], ['filepath65', 51], ['filepath66', 54], ['filepath67', 69], ['filepath68', 61], ['filepath69', 74], ['filepath70', 40], ['filepath71', 80], ['filepath72', 47], ['filepath73', 47], ['filepath74', 38], ['filepath75', 47], ['filepath76', 39], ['filepath77', 90], ['filepath78', 83], ['filepath79', 32], ['filepath80', 18], ['filepath81', 77], ['filepath82', 89], ['filepath83', 83], ['filepath84', 3], ['filepath85', 49], ['filepath86', 64], ['filepath87', 2], ['filepath88', 7], ['filepath89', 73], ['filepath90', 21], ['filepath91', 67], ['filepath92', 64], ['filepath93', 93], ['filepath94', 0], ['filepath95', 29], ['filepath96', 60], ['filepath97', 45], ['filepath98', 13], ['filepath99', 40], ['filepath100', 93], ['filepath101', 1], ['filepath102', 47], ['filepath103', 61], ['filepath104', 29], ['filepath105', 47], ['filepath106', 15], ['filepath107', 28], ['filepath108', 39], ['filepath109', 35], ['filepath110', 9], ['filepath111', 64], ['filepath112', 99], ['filepath113', 91], ['filepath114', 87], ['filepath115', 29], ['filepath116', 90], ['filepath117', 28], ['filepath118', 6], ['filepath119', 68], ['filepath120', 38], ['filepath121', 31], ['filepath122', 64], ['filepath123', 14], ['filepath124', 22], ['filepath125', 45], ['filepath126', 17], ['filepath127', 55], ['filepath128', 19], ['filepath129', 63], ['filepath130', 4], ['filepath131', 10], ['filepath132', 13], ['filepath133', 87], ['filepath134', 4], ['filepath135', 66], ['filepath136', 85], ['filepath137', 56], ['filepath138', 18], ['filepath139', 88], ['filepath140', 69], ['filepath141', 96], ['filepath142', 52], ['filepath143', 20], ['filepath144', 32], ['filepath145', 86], ['filepath146', 1], ['filepath147', 69], ['filepath148', 23], ['filepath149', 95], ['filepath150', 45], ['filepath151', 44], ['filepath152', 100], ['filepath153', 50], ['filepath154', 66], ['filepath155', 49], ['filepath156', 56], ['filepath157', 51], ['filepath158', 52], ['filepath159', 11], ['filepath160', 29], ['filepath161', 41], ['filepath162', 9], ['filepath163', 86], ['filepath164', 100], ['filepath165', 18], ['filepath166', 63], ['filepath167', 98], ['filepath168', 24], ['filepath169', 84], ['filepath170', 19], ['filepath171', 79], ['filepath172', 93], ['filepath173', 63], ['filepath174', 72], ['filepath175', 15], ['filepath176', 30], ['filepath177', 76], ['filepath178', 38], ['filepath179', 57], ['filepath180', 48], ['filepath181', 78], ['filepath182', 98], ['filepath183', 72], ['filepath184', 51], ['filepath185', 92], ['filepath186', 64], ['filepath187', 10], ['filepath188', 48], ['filepath189', 34], ['filepath190', 62], ['filepath191', 41], ['filepath192', 69], ['filepath193', 37], ['filepath194', 3], ['filepath195', 19], ['filepath196', 81], ['filepath197', 54], ['filepath198', 25], ['filepath199', 62], ['filepath200', 39], ['filepath201', 61], ['filepath202', 69], ['filepath203', 80], ['filepath204', 8], ['filepath205', 18], ['filepath206', 10], ['filepath207', 8], ['filepath208', 48], ['filepath209', 20], ['filepath210', 84], ['filepath211', 11], ['filepath212', 92], ['filepath213', 91], ['filepath214', 53], ['filepath215', 47], ['filepath216', 52], ['filepath217', 66], ['filepath218', 14], ['filepath219', 82], ['filepath220', 96], ['filepath221', 59], ['filepath222', 77], ['filepath223', 22], ['filepath224', 29], ['filepath225', 96], ['filepath226', 34], ['filepath227', 87], ['filepath228', 47], ['filepath229', 8], ['filepath230', 12], ['filepath231', 6], ['filepath232', 46], ['filepath233', 82], ['filepath234', 64], ['filepath235', 64], ['filepath236', 56], ['filepath237', 81], ['filepath238', 39], ['filepath239', 48], ['filepath240', 77], ['filepath241', 18], ['filepath242', 41], ['filepath243', 72], ['filepath244', 7], ['filepath245', 54], ['filepath246', 78], ['filepath247', 92], ['filepath248', 100], ['filepath249', 76], ['filepath250', 60], ['filepath251', 0], ['filepath252', 2], ['filepath253', 27], ['filepath254', 35], ['filepath255', 80], ['filepath256', 21], ['filepath257', 34], ['filepath258', 94], ['filepath259', 32], ['filepath260', 5], ['filepath261', 90], ['filepath262', 57], ['filepath263', 81], ['filepath264', 32], ['filepath265', 45], ['filepath266', 29], ['filepath267', 80], ['filepath268', 18], ['filepath269', 91], ['filepath270', 9], ['filepath271', 30], ['filepath272', 13], ['filepath273', 13], ['filepath274', 33], ['filepath275', 35], ['filepath276', 65], ['filepath277', 63], ['filepath278', 33], ['filepath279', 66], ['filepath280', 4], ['filepath281', 52], ['filepath282', 47], ['filepath283', 72], ['filepath284', 99], ['filepath285', 96], ['filepath286', 48], ['filepath287', 21], ['filepath288', 62], ['filepath289', 2], ['filepath290', 36], ['filepath291', 36], ['filepath292', 33], ['filepath293', 26], ['filepath294', 34], ['filepath295', 19], ['filepath296', 85], ['filepath297', 20], ['filepath298', 5], ['filepath299', 37], ['filepath300', 77], ['filepath301', 57], ['filepath302', 46], ['filepath303', 21], ['filepath304', 3], ['filepath305', 69], ['filepath306', 25], ['filepath307', 41], ['filepath308', 5], ['filepath309', 20], ['filepath310', 91], ['filepath311', 57], ['filepath312', 35], ['filepath313', 37], ['filepath314', 53], ['filepath315', 72], ['filepath316', 87], ['filepath317', 87], ['filepath318', 53], ['filepath319', 50], ['filepath320', 99], ['filepath321', 34], ['filepath322', 65], ['filepath323', 44], ['filepath324', 53], ['filepath325', 47], ['filepath326', 5], ['filepath327', 27], ['filepath328', 12], ['filepath329', 58], ['filepath330', 20], ['filepath331', 56], ['filepath332', 92], ['filepath333', 32], ['filepath334', 81], ['filepath335', 84], ['filepath336', 63], ['filepath337', 93], ['filepath338', 75], ['filepath339', 22], ['filepath340', 47], ['filepath341', 83], ['filepath342', 25], ['filepath343', 13], ['filepath344', 11], ['filepath345', 63], ['filepath346', 65], ['filepath347', 14], ['filepath348', 9], ['filepath349', 75], ['filepath350', 10], ['filepath351', 0], ['filepath352', 90], ['filepath353', 11], ['filepath354', 64], ['filepath355', 81], ['filepath356', 98], ['filepath357', 21], ['filepath358', 3], ['filepath359', 0], ['filepath360', 48], ['filepath361', 24], ['filepath362', 35], ['filepath363', 53], ['filepath364', 1], ['filepath365', 8], ['filepath366', 76], ['filepath367', 88], ['filepath368', 70], ['filepath369', 62], ['filepath370', 90], ['filepath371', 60], ['filepath372', 8], ['filepath373', 85], ['filepath374', 8], ['filepath375', 2], ['filepath376', 87], ['filepath377', 11], ['filepath378', 44], ['filepath379', 87], ['filepath380', 58], ['filepath381', 70], ['filepath382', 9], ['filepath383', 72], ['filepath384', 82], ['filepath385', 35], ['filepath386', 34], ['filepath387', 75], ['filepath388', 48], ['filepath389', 64], ['filepath390', 83], ['filepath391', 32], ['filepath392', 7], ['filepath393', 93], ['filepath394', 51], ['filepath395', 65], ['filepath396', 5], ['filepath397', 63], ['filepath398', 65], ['filepath399', 16], ['filepath400', 51], ['filepath401', 21], ['filepath402', 23], ['filepath403', 14], ['filepath404', 40], ['filepath405', 22], ['filepath406', 81], ['filepath407', 16], ['filepath408', 77], ['filepath409', 75], ['filepath410', 34], ['filepath411', 97], ['filepath412', 49], ['filepath413', 60], ['filepath414', 27], ['filepath415', 3], ['filepath416', 9], ['filepath417', 90], ['filepath418', 15], ['filepath419', 96], ['filepath420', 77], ['filepath421', 96], ['filepath422', 29], ['filepath423', 77], ['filepath424', 30], ['filepath425', 47], ['filepath426', 16], ['filepath427', 28], ['filepath428', 14], ['filepath429', 30], ['filepath430', 74], ['filepath431', 8], ['filepath432', 67], ['filepath433', 78], ['filepath434', 8], ['filepath435', 56], ['filepath436', 15], ['filepath437', 25], ['filepath438', 63], ['filepath439', 79], ['filepath440', 51], ['filepath441', 10], ['filepath442', 97], ['filepath443', 95], ['filepath444', 36], ['filepath445', 48], ['filepath446', 97], ['filepath447', 84], ['filepath448', 12], ['filepath449', 84], ['filepath450', 6], ['filepath451', 66], ['filepath452', 93], ['filepath453', 57], ['filepath454', 54], ['filepath455', 94], ['filepath456', 27], ['filepath457', 88], ['filepath458', 87], ['filepath459', 21], ['filepath460', 51], ['filepath461', 73], ['filepath462', 35], ['filepath463', 94], ['filepath464', 32], ['filepath465', 58], ['filepath466', 69], ['filepath467', 24], ['filepath468', 48], ['filepath469', 59], ['filepath470', 93], ['filepath471', 35], ['filepath472', 81], ['filepath473', 62], ['filepath474', 62], ['filepath475', 89], ['filepath476', 97], ['filepath477', 29], ['filepath478', 8], ['filepath479', 51], ['filepath480', 54], ['filepath481', 89], ['filepath482', 61], ['filepath483', 75], ['filepath484', 82], ['filepath485', 46], ['filepath486', 35], ['filepath487', 69], ['filepath488', 34], ['filepath489', 38], ['filepath490', 88], ['filepath491', 71], ['filepath492', 7], ['filepath493', 76], ['filepath494', 37], ['filepath495', 48], ['filepath496', 70], ['filepath497', 7], ['filepath498', 58], ['filepath499', 76], ['filepath500', 19]]


filepath_to_check = "filepath500"

# if(filepath_to_check == item[0] for item in filepaths_array):
#     print(item)

for item in test_array:
    if filepath_to_check == item[0]:
        item[1] += 1
        print(item)

print(test_array[499])
# print(f"文件路径 {filepath_to_check} 是否在数组中: {is_in_array}")

['filepath500', 20]
['filepath500', 20]


In [18]:
filepath_to_check = "filepath501"
for item in test_array:
    if filepath_to_check == item[0]:
        item[1] += 1
        # print(item)
        break
else:
    test_array.append([filepath_to_check,0])

In [19]:
test_array

[['filepath1', 29],
 ['filepath2', 50],
 ['filepath3', 20],
 ['filepath4', 96],
 ['filepath5', 29],
 ['filepath6', 14],
 ['filepath7', 47],
 ['filepath8', 7],
 ['filepath9', 68],
 ['filepath10', 48],
 ['filepath11', 89],
 ['filepath12', 12],
 ['filepath13', 7],
 ['filepath14', 38],
 ['filepath15', 24],
 ['filepath16', 21],
 ['filepath17', 2],
 ['filepath18', 86],
 ['filepath19', 57],
 ['filepath20', 58],
 ['filepath21', 57],
 ['filepath22', 2],
 ['filepath23', 82],
 ['filepath24', 39],
 ['filepath25', 1],
 ['filepath26', 98],
 ['filepath27', 100],
 ['filepath28', 97],
 ['filepath29', 91],
 ['filepath30', 1],
 ['filepath31', 78],
 ['filepath32', 70],
 ['filepath33', 82],
 ['filepath34', 70],
 ['filepath35', 74],
 ['filepath36', 84],
 ['filepath37', 25],
 ['filepath38', 40],
 ['filepath39', 2],
 ['filepath40', 40],
 ['filepath41', 87],
 ['filepath42', 59],
 ['filepath43', 47],
 ['filepath44', 7],
 ['filepath45', 87],
 ['filepath46', 81],
 ['filepath47', 72],
 ['filepath48', 42],
 ['filep